In [14]:
### Import libraries ###
import os
import pickle
import numpy as np
import pandas as pd

In [15]:
Sim0 = "/Users/simondn/Documents/RashomonActiveLearning/Results/BostonHousing/TreeFarms/Raw/Seed0_DataBostonHousing_TP0.2_CP0.8_STTreeEnsembleQBC_MTTreeFarms_UEI0_NE100_Reg0.01_RBA0.01.pkl"
Sim1 = "/Users/simondn/Documents/RashomonActiveLearning/Results/BostonHousing/TreeFarms/Raw/Seed1_DataBostonHousing_TP0.2_CP0.8_STTreeEnsembleQBC_MTTreeFarms_UEI0_NE100_Reg0.01_RBA0.01.pkl"
Sim2 = "/Users/simondn/Documents/RashomonActiveLearning/Results/BostonHousing/TreeFarms/Raw/Seed2_DataBostonHousing_TP0.2_CP0.8_STTreeEnsembleQBC_MTTreeFarms_UEI0_NE100_Reg0.01_RBA0.01.pkl"


In [16]:
with open(Sim0, "rb") as f:
    data0 = pickle.load(f)
with open(Sim1, "rb") as f:
    data1 = pickle.load(f)
with open(Sim2, "rb") as f:
    data2 = pickle.load(f)

In [17]:
print("Seed0:")
print("SelectionHistory:" , len(data0["SelectionHistory"]))
print("ErrorVec:" , len(data0["ErrorVec"]))
print("AllErrorVec:" , len(data0["AllErrorVec"]))
print("UniqueModelsInRashomonSet:" , len(data0["RashomonCommitteeDict"]["UniqueModelsInRashomonSet"]))
print("AllModelsInRashomonSet:" , len(data0["RashomonCommitteeDict"]["AllModelsInRashomonSet"]))

Seed0:
SelectionHistory: 324
ErrorVec: 324
AllErrorVec: 324
UniqueModelsInRashomonSet: 324
AllModelsInRashomonSet: 324


In [18]:
data1["SimulationParameters"]

{'DataFileInput': 'BostonHousing',
 'Seed': '1',
 'TestProportion': '0.2',
 'CandidateProportion': '0.8',
 'SelectorType': 'TreeEnsembleQBCFunction',
 'ModelType': 'TreeFarmsFunction',
 'UniqueErrorsInput': '0',
 'n_estimators': '100',
 'regularization': '0.01',
 'rashomon_bound_adder': '0.01',
 'Type': 'Classification'}

In [19]:
print("Seed1")
print("SelectionHistory:" , len(data1["SelectionHistory"]))
print("ErrorVec:" , len(data1["ErrorVec"]))
print("AllErrorVec:" , len(data1["AllErrorVec"]))
print("UniqueModelsInRashomonSet:" , len(data1["RashomonCommitteeDict"]["UniqueModelsInRashomonSet"]))
print("AllModelsInRashomonSet:" , len(data1["RashomonCommitteeDict"]["AllModelsInRashomonSet"]))

Seed1
SelectionHistory: 324
ErrorVec: 324
AllErrorVec: 324
UniqueModelsInRashomonSet: 324
AllModelsInRashomonSet: 324


In [20]:
print("Seed2")
print("SelectionHistory:" , len(data2["SelectionHistory"]))
print("ErrorVec:" , len(data2["ErrorVec"]))
print("AllErrorVec:" , len(data2["AllErrorVec"]))
print("UniqueModelsInRashomonSet:" , len(data2["RashomonCommitteeDict"]["UniqueModelsInRashomonSet"]))
print("AllModelsInRashomonSet:" , len(data2["RashomonCommitteeDict"]["AllModelsInRashomonSet"]))

Seed2
SelectionHistory: 324
ErrorVec: 324
AllErrorVec: 324
UniqueModelsInRashomonSet: 324
AllModelsInRashomonSet: 324


In [21]:
data1AllErrorVec = data1["AllErrorVec"]

In [22]:
data1ErrorVec = data1["ErrorVec"]